# Libraries requirement

In [ ]:
# numpy
# scipy
# pandas
# sklearn
# openbabel
# torch
# torch_geometric
# numba
# dscribe
# networkX

# Imports

In [ ]:
from common import *
from babel_113_half_bonds import compute_features, ATOM_FEATURES_COLUMNS, BOND_FEATURES_COLUMNS, GLOBAL_FEATURES_COLUMNS
import itertools
import scipy.linalg
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import os
import numba
from dscribe.descriptors import ACSF
from dscribe.core.system import System
from scipy.spatial.transform import Rotation as R

# Read data

In [ ]:
script_dir = os.path.abspath(os.path.dirname(__file__))

In [ ]:
nodes = pd.read_csv(script_dir + '/../../data/structures.csv')
train = pd.read_csv(script_dir + '/../../data/train.csv')
test = pd.read_csv(script_dir + '/../../data/test.csv')

# Public kernels references to download data or where code was extracted

In [ ]:
#https://www.kaggle.com/scaomath/parallelization-of-coulomb-yukawa-interaction
#https://www.kaggle.com/scaomath/no-memory-reduction-workflow-for-each-type-lb-1-28
#https://www.kaggle.com/scaomath/giba-molecular-features
#https://github.com/Kaggle/kaggle-api
#pip install kaggle --upgrade
#kaggle datasets download scaomath/giba-molecular-features
#kaggle kernels output zaharch/quantum-machine-9-qm9

# Vizualization of the kept features selected with a previous permutation importance

In [ ]:
import pandas as pd
permute_results = pd.read_pickle(script_dir + '/processed_data/permute_results.9Y.pkl')
features = permute_results.loc[permute_results['loss'] > -2.09]
features = features.loc[~features['feature_name'].str.contains('dist_H')]
features

# Build the dataset for a subsample of train molecules (Optionnal)

In [ ]:
molecules = list(train['molecule_name'].unique()[:4]) + list(train['molecule_name'].iloc[100:].sample(0).unique())
print(f'subsampled {len(molecules)} molecules')
nodes = nodes.loc[nodes['molecule_name'].isin(molecules)]
train = train.loc[train['molecule_name'].isin(molecules)]
test = test.loc[test['molecule_name'].isin(molecules)]

## Utils

In [ ]:
def apply_random_rotation(vectors):
    r = R.from_euler('zxy', np.random.random(size = (1, 3)) * 360, degrees=True)
    return r.apply(vectors)

SYMBOL=['H', 'C', 'N', 'O', 'F']

ACSF_GENERATOR = ACSF(
    species=SYMBOL,
    rcut=6.0,
    g2_params=[[1, 1], [1, 2], [1, 3]],
    g4_params=[[1, 1, 1], [1, 2, 1], [1, 1, -1], [1, 2, -1]],
)

In [ ]:
@numba.njit
def build_bond_vector(connectivity, xyz):
    bond_vectors = np.zeros((connectivity.shape[0], 3))
    
    for bond_i in range(connectivity.shape[0]):
        atom_i, atom_j = connectivity[bond_i, 0], connectivity[bond_i, 1]
        bond_vectors[bond_i] = xyz[atom_j] - xyz[atom_j]
        bond_i += 1

    return bond_vectors

## Datasets checks

In [ ]:
nodes.head()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train['molecule_name'].isin(test['molecule_name']).sum()

In [ ]:
train[['molecule_name', 'atom_index_0', 'atom_index_1']].drop_duplicates().shape

In [ ]:
train.shape

# Transform for easier storage first

## Merge train and test

In [ ]:
train['dataset'] = 'train'
test['dataset'] = 'test'

In [ ]:
partial_edges = pd.concat([train, test], axis = 0, sort = False).reset_index(drop = True)

In [ ]:
partial_edges.shape

## Check

In [ ]:
partial_edges.sample(n = 10)

In [ ]:
partial_edges.head(10)

In [ ]:
nodes['molecule_name'].isin(partial_edges['molecule_name']).mean()

In [ ]:
partial_edges['molecule_name'].isin(nodes['molecule_name']).mean()

## Make full edges

In [ ]:
atom_count = nodes.groupby('molecule_name')['atom_index'].count()
molecules = list(atom_count.index)

In [ ]:
atom_count.head()

In [ ]:
atom_count = atom_count.values

In [ ]:
atom_count

In [ ]:
@numba.njit
def build_all_edges(molecule_atom_count):
    full_count = int((molecule_atom_count * (molecule_atom_count - 1) // 2).sum())
    all_edges = np.zeros((full_count, 3), dtype = np.int64)
    
    edge_i = 0
    for molecule_i, atom_count in enumerate(molecule_atom_count):
        for atom_i in range(atom_count):
            for atom_j in range(atom_count):
                if atom_i < atom_j:
                    all_edges[edge_i, 0] = molecule_i
                    all_edges[edge_i, 1] = atom_i
                    all_edges[edge_i, 2] = atom_j
                    edge_i += 1
    
    assert edge_i == all_edges.shape[0]
    return all_edges

In [ ]:
all_edges = build_all_edges(atom_count)

## Make babel for atoms, bonds, and global

In [ ]:
all_global_features = []
all_atom_features = []
all_bond_features = []

In [ ]:
for molecule_i, molecule in tqdm.tqdm_notebook(list(enumerate(molecules))):
    global_features, atom_features, bond_features = compute_features(molecule)
    
    global_features = np.concatenate([np.ones((1, 1)) * molecule_i, global_features.reshape(1, -1)], axis = 1)
    atom_features = np.concatenate([np.ones((atom_features.shape[0], 1)) * molecule_i, atom_features], axis = 1)
    bond_features = np.concatenate([np.ones((bond_features.shape[0], 1)) * molecule_i, bond_features], axis = 1)
    
    all_global_features.append(global_features)
    all_atom_features.append(atom_features)
    all_bond_features.append(bond_features)

In [ ]:
all_global_features = np.concatenate(all_global_features, axis = 0)
all_atom_features = np.concatenate(all_atom_features, axis = 0)
all_bond_features = np.concatenate(all_bond_features, axis = 0)

In [ ]:
all_global_features.shape

In [ ]:
all_atom_features.shape

In [ ]:
all_bond_features.shape

In [ ]:
all_bond_features = pd.DataFrame(all_bond_features, columns = ['molecule_id'] + BOND_FEATURES_COLUMNS)

In [ ]:
assert all_edges.shape[0] == all_bond_features.shape[0]
np.testing.assert_array_equal(all_edges[:, 0], all_edges[:, 0])

In [ ]:
all_global_features = pd.DataFrame(all_global_features, columns = ['molecule_id'] + GLOBAL_FEATURES_COLUMNS)

In [ ]:
all_atom_features = pd.DataFrame(all_atom_features, columns = ['molecule_id'] + ATOM_FEATURES_COLUMNS)

In [ ]:
all_global_features['molecule_id'] = all_global_features['molecule_id'].astype(int)
all_bond_features['molecule_id'] = all_bond_features['molecule_id'].astype(int)
all_bond_features['atom_index_0'] = all_edges[:, 1]
all_bond_features['atom_index_1'] = all_edges[:, 2]
all_atom_features['molecule_id'] = all_atom_features['molecule_id'].astype(int)

## Add atom id and positions

In [ ]:
indexed_nodes = nodes.set_index('molecule_name')

In [ ]:
positions = indexed_nodes.loc[molecules, ['x', 'y', 'z']].values

In [ ]:
all_atom_features['x'] = positions[:, 0]
all_atom_features['y'] = positions[:, 1]
all_atom_features['z'] = positions[:, 2]
all_atom_features['atom'] = indexed_nodes.loc[molecules, 'atom'].values
all_atom_features['atom_index'] = indexed_nodes.loc[molecules, 'atom_index'].values

## Add target types

In [ ]:
molecules_index = pd.Series(molecules, name = 'molecule_name').to_frame()
molecules_index['molecule_id'] = np.arange(molecules_index.shape[0], dtype = int)
molecules_index.head()

In [ ]:
partial_edges.shape

In [ ]:
partial_edges = pd.merge(partial_edges, molecules_index, on = 'molecule_name', how = 'left')

In [ ]:
partial_edges.shape

In [ ]:
partial_edges['molecule_id'].isnull().sum()

In [ ]:
partial_edges.head()

In [ ]:
all_bond_features.head()

In [ ]:
merge_1 = pd.merge(all_bond_features[['molecule_id', 'atom_index_0', 'atom_index_1']], partial_edges[['molecule_id', 'atom_index_0', 'atom_index_1', 'type', 'scalar_coupling_constant', 'dataset']], left_on = ['molecule_id', 'atom_index_0', 'atom_index_1'], right_on = ['molecule_id', 'atom_index_0', 'atom_index_1'], how = 'left')[['type', 'scalar_coupling_constant', 'dataset']]
merge_2 = pd.merge(all_bond_features[['molecule_id', 'atom_index_0', 'atom_index_1']], partial_edges[['molecule_id', 'atom_index_0', 'atom_index_1', 'type', 'scalar_coupling_constant', 'dataset']], left_on = ['molecule_id', 'atom_index_0', 'atom_index_1'], right_on = ['molecule_id', 'atom_index_1', 'atom_index_0'], how = 'left')[['type', 'scalar_coupling_constant', 'dataset']]

In [ ]:
all_bond_features.shape

In [ ]:
all_bond_features.head()

In [ ]:
for c in ['type', 'scalar_coupling_constant', 'dataset']:
    all_bond_features[c] = np.nan
    all_bond_features.loc[all_bond_features[c].isnull(), c] = merge_1[c]
    all_bond_features.loc[all_bond_features[c].isnull(), c] = merge_2[c]

In [ ]:
all_bond_features.loc[all_bond_features['type'].isnull(), 'type'] = 'VOID'
all_bond_features.loc[all_bond_features['scalar_coupling_constant'].isnull(), 'scalar_coupling_constant'] = -1

## Add kernel atom features

### Add computationnal features

In [ ]:
def map_atom_info(df_1, df_2, atom_idx):
    df = pd.merge(df_1, df_2, how = 'left',
                  left_on  = ['molecule_id', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_id',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    return df

    
def find_dist(df):
    df_p_0 = df[['x_0', 'y_0', 'z_0']].values
    df_p_1 = df[['x_1', 'y_1', 'z_1']].values
    
    df['dist'] = np.linalg.norm(df_p_0 - df_p_1, axis=1)
    df['dist_inv2'] = 1/df['dist']**2
    df['dist_x'] = (df['x_0'] - df['x_1']) ** 2
    df['dist_y'] = (df['y_0'] - df['y_1']) ** 2
    df['dist_z'] = (df['z_0'] - df['z_1']) ** 2
    return df

def find_closest_atom(df):    
    df_temp = df.loc[:,["molecule_id",
                      "atom_index_0","atom_index_1",
                      "dist","x_0","y_0","z_0","x_1","y_1","z_1"]].copy()
    df_temp_ = df_temp.copy()
    df_temp_ = df_temp_.rename(columns={'atom_index_0': 'atom_index_1',
                                       'atom_index_1': 'atom_index_0',
                                       'x_0': 'x_1',
                                       'y_0': 'y_1',
                                       'z_0': 'z_1',
                                       'x_1': 'x_0',
                                       'y_1': 'y_0',
                                       'z_1': 'z_0'})
    df_temp_all = pd.concat((df_temp,df_temp_),axis=0)

    df_temp_all["min_distance"]=df_temp_all.groupby(['molecule_id', 
                                                     'atom_index_0'])['dist'].transform('min')
    df_temp_all["max_distance"]=df_temp_all.groupby(['molecule_id', 
                                                     'atom_index_0'])['dist'].transform('max')
    
    df_temp = df_temp_all[df_temp_all["min_distance"]==df_temp_all["dist"]].copy()
    df_temp = df_temp.drop(['x_0','y_0','z_0','min_distance'], axis=1)
    df_temp = df_temp.rename(columns={'atom_index_0': 'atom_index',
                                         'atom_index_1': 'atom_index_closest',
                                         'dist': 'distance_closest',
                                         'x_1': 'x_closest',
                                         'y_1': 'y_closest',
                                         'z_1': 'z_closest'})
    df_temp = df_temp.drop_duplicates(subset=['molecule_id', 'atom_index'])
    
    for atom_idx in [0,1]:
        df = map_atom_info(df,df_temp, atom_idx)
        df = df.rename(columns={'atom_index_closest': f'atom_index_closest_{atom_idx}',
                                        'distance_closest': f'distance_closest_{atom_idx}',
                                        'x_closest': f'x_closest_{atom_idx}',
                                        'y_closest': f'y_closest_{atom_idx}',
                                        'z_closest': f'z_closest_{atom_idx}'})
        
    df_temp= df_temp_all[df_temp_all["max_distance"]==df_temp_all["dist"]].copy()
    df_temp = df_temp.drop(['x_0','y_0','z_0','max_distance'], axis=1)
    df_temp= df_temp.rename(columns={'atom_index_0': 'atom_index',
                                         'atom_index_1': 'atom_index_farthest',
                                         'dist': 'distance_farthest',
                                         'x_1': 'x_farthest',
                                         'y_1': 'y_farthest',
                                         'z_1': 'z_farthest'})
    df_temp = df_temp.drop_duplicates(subset=['molecule_id', 'atom_index'])
        
    for atom_idx in [0,1]:
        df = map_atom_info(df,df_temp, atom_idx)
        df = df.rename(columns={'atom_index_farthest': f'atom_index_farthest_{atom_idx}',
                                        'distance_farthest': f'distance_farthest_{atom_idx}',
                                        'x_farthest': f'x_farthest_{atom_idx}',
                                        'y_farthest': f'y_farthest_{atom_idx}',
                                        'z_farthest': f'z_farthest_{atom_idx}'})
    return df


def add_cos_features(df):
    
    df["distance_center0"] = np.sqrt((df['x_0']-df['c_x'])**2 \
                                   + (df['y_0']-df['c_y'])**2 \
                                   + (df['z_0']-df['c_z'])**2)
    df["distance_center1"] = np.sqrt((df['x_1']-df['c_x'])**2 \
                                   + (df['y_1']-df['c_y'])**2 \
                                   + (df['z_1']-df['c_z'])**2)
    
    df['distance_c0'] = np.sqrt((df['x_0']-df['x_closest_0'])**2 + \
                                (df['y_0']-df['y_closest_0'])**2 + \
                                (df['z_0']-df['z_closest_0'])**2)
    df['distance_c1'] = np.sqrt((df['x_1']-df['x_closest_1'])**2 + \
                                (df['y_1']-df['y_closest_1'])**2 + \
                                (df['z_1']-df['z_closest_1'])**2)
    
    df["distance_f0"] = np.sqrt((df['x_0']-df['x_farthest_0'])**2 + \
                                (df['y_0']-df['y_farthest_0'])**2 + \
                                (df['z_0']-df['z_farthest_0'])**2)
    df["distance_f1"] = np.sqrt((df['x_1']-df['x_farthest_1'])**2 + \
                                (df['y_1']-df['y_farthest_1'])**2 + \
                                (df['z_1']-df['z_farthest_1'])**2)
    
    vec_center0_x = (df['x_0']-df['c_x'])/(df["distance_center0"]+1e-10)
    vec_center0_y = (df['y_0']-df['c_y'])/(df["distance_center0"]+1e-10)
    vec_center0_z = (df['z_0']-df['c_z'])/(df["distance_center0"]+1e-10)
    
    vec_center1_x = (df['x_1']-df['c_x'])/(df["distance_center1"]+1e-10)
    vec_center1_y = (df['y_1']-df['c_y'])/(df["distance_center1"]+1e-10)
    vec_center1_z = (df['z_1']-df['c_z'])/(df["distance_center1"]+1e-10)
    
    vec_c0_x = (df['x_0']-df['x_closest_0'])/(df["distance_c0"]+1e-10)
    vec_c0_y = (df['y_0']-df['y_closest_0'])/(df["distance_c0"]+1e-10)
    vec_c0_z = (df['z_0']-df['z_closest_0'])/(df["distance_c0"]+1e-10)
    
    vec_c1_x = (df['x_1']-df['x_closest_1'])/(df["distance_c1"]+1e-10)
    vec_c1_y = (df['y_1']-df['y_closest_1'])/(df["distance_c1"]+1e-10)
    vec_c1_z = (df['z_1']-df['z_closest_1'])/(df["distance_c1"]+1e-10)
    
    vec_f0_x = (df['x_0']-df['x_farthest_0'])/(df["distance_f0"]+1e-10)
    vec_f0_y = (df['y_0']-df['y_farthest_0'])/(df["distance_f0"]+1e-10)
    vec_f0_z = (df['z_0']-df['z_farthest_0'])/(df["distance_f0"]+1e-10)
    
    vec_f1_x = (df['x_1']-df['x_farthest_1'])/(df["distance_f1"]+1e-10)
    vec_f1_y = (df['y_1']-df['y_farthest_1'])/(df["distance_f1"]+1e-10)
    vec_f1_z = (df['z_1']-df['z_farthest_1'])/(df["distance_f1"]+1e-10)
    
    vec_x = (df['x_1']-df['x_0'])/df['dist']
    vec_y = (df['y_1']-df['y_0'])/df['dist']
    vec_z = (df['z_1']-df['z_0'])/df['dist']
    
    df["cos_c0_c1"] = vec_c0_x*vec_c1_x + vec_c0_y*vec_c1_y + vec_c0_z*vec_c1_z
    df["cos_f0_f1"] = vec_f0_x*vec_f1_x + vec_f0_y*vec_f1_y + vec_f0_z*vec_f1_z
    
    df["cos_c0_f0"] = vec_c0_x*vec_f0_x + vec_c0_y*vec_f0_y + vec_c0_z*vec_f0_z
    df["cos_c1_f1"] = vec_c1_x*vec_f1_x + vec_c1_y*vec_f1_y + vec_c1_z*vec_f1_z
    
    df["cos_center0_center1"] = vec_center0_x*vec_center1_x \
                              + vec_center0_y*vec_center1_y \
                              + vec_center0_z*vec_center1_z
    
    df["cos_c0"] = vec_c0_x*vec_x + vec_c0_y*vec_y + vec_c0_z*vec_z
    df["cos_c1"] = vec_c1_x*vec_x + vec_c1_y*vec_y + vec_c1_z*vec_z
    
    df["cos_f0"] = vec_f0_x*vec_x + vec_f0_y*vec_y + vec_f0_z*vec_z
    df["cos_f1"] = vec_f1_x*vec_x + vec_f1_y*vec_y + vec_f1_z*vec_z
    
    df["cos_center0"] = vec_center0_x*vec_x + vec_center0_y*vec_y + vec_center0_z*vec_z
    df["cos_center1"] = vec_center1_x*vec_x + vec_center1_y*vec_y + vec_center1_z*vec_z

    return df

def add_dist_features(df):
    # Andrew's features selected
    df[f'molecule_atom_index_0_dist_mean'] = df.groupby(['molecule_id', 'atom_index_0'])['dist'].transform('mean')
    df[f'molecule_atom_index_0_dist_mean_diff'] = df[f'molecule_atom_index_0_dist_mean'] - df['dist']
    df[f'molecule_atom_index_0_dist_min'] = df.groupby(['molecule_id', 'atom_index_0'])['dist'].transform('min')
    df[f'molecule_atom_index_0_dist_min_diff'] = df[f'molecule_atom_index_0_dist_min'] - df['dist']
    df[f'molecule_atom_index_0_dist_std'] = df.groupby(['molecule_id', 'atom_index_0'])['dist'].transform('std')

    df[f'molecule_atom_index_1_dist_mean'] = df.groupby(['molecule_id', 'atom_index_1'])['dist'].transform('mean')
    df[f'molecule_atom_index_1_dist_mean_diff'] = df[f'molecule_atom_index_1_dist_mean'] - df['dist']
    df[f'molecule_atom_index_1_dist_min'] = df.groupby(['molecule_id', 'atom_index_1'])['dist'].transform('min')
    df[f'molecule_atom_index_1_dist_min_diff'] = df[f'molecule_atom_index_1_dist_min'] - df['dist']
    df[f'molecule_atom_index_1_dist_std'] = df.groupby(['molecule_id', 'atom_index_1'])['dist'].transform('std')
    
    df[f'molecule_type_dist_mean'] = df.groupby(['molecule_id', 'type'])['dist'].transform('mean')
    df[f'molecule_type_dist_mean_diff'] = df[f'molecule_type_dist_mean'] - df['dist']
    
    return df

def get_features(df, struct):
    struct = struct[['molecule_id', 'atom_index', 'x', 'y', 'z']].copy()
    for atom_idx in [0,1]:
        df = map_atom_info(df, struct, atom_idx)
        df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
        struct['c_x'] = struct.groupby('molecule_id')['x'].transform('mean')
        struct['c_y'] = struct.groupby('molecule_id')['y'].transform('mean')
        struct['c_z'] = struct.groupby('molecule_id')['z'].transform('mean')
    
    print('df = find_dist(df)')
    df = find_dist(df)
    print('df = find_closest_atom(df)')
    df = find_closest_atom(df)
    print('df = add_cos_features(df)')
    df = add_cos_features(df)
    print('df = add_dist_features(df)')
    df = add_dist_features(df)
    
    return df

In [ ]:
computationnal_columns = ['dist_inv2',
 'distance_closest_0',
 'distance_closest_1',
 'distance_farthest_0',
 'distance_farthest_1',
 'cos_c0_c1', 'cos_f0_f1','cos_c0_f0', 'cos_c1_f1',
 'cos_center0_center1', 'cos_c0', 'cos_c1', 'cos_f0', 'cos_f1',
 'cos_center0', 'cos_center1',
 'molecule_atom_index_0_dist_mean',
 'molecule_atom_index_0_dist_mean_diff',
 'molecule_atom_index_0_dist_min',
 'molecule_atom_index_0_dist_min_diff',
 'molecule_atom_index_0_dist_std',
 'molecule_atom_index_1_dist_mean',
 'molecule_atom_index_1_dist_mean_diff',
 'molecule_atom_index_1_dist_min',
 'molecule_atom_index_1_dist_min_diff',
 'molecule_atom_index_1_dist_std',
 'molecule_type_dist_mean',
 'molecule_type_dist_mean_diff']

In [ ]:
bond_features_view = all_bond_features[['molecule_id', 'atom_index_0', 'atom_index_1', 'type']].copy()

In [ ]:
new_features = get_features(bond_features_view, all_atom_features)

In [ ]:
new_features.shape

In [ ]:
all_bond_features = pd.concat([all_bond_features, new_features[computationnal_columns]], axis = 1, sort = False)

In [ ]:
all_bond_features.shape

In [ ]:
BOND_FEATURES_COLUMNS += computationnal_columns

set(e.columns) - set(computationnal_columns)

### Fill na

In [ ]:
all_atom_features = all_atom_features.fillna(0.0)
all_bond_features = all_bond_features.fillna(0.0)

## Add bond distances

In [ ]:
bond_xyz_0 = pd.merge(all_bond_features[['molecule_id', 'atom_index_0']], all_atom_features[['molecule_id', 'atom_index', 'x', 'y', 'z']], left_on = ['molecule_id', 'atom_index_0'], right_on = ['molecule_id', 'atom_index'], how = 'left')[['x', 'y', 'z']]
bond_xyz_1 = pd.merge(all_bond_features[['molecule_id', 'atom_index_1']], all_atom_features[['molecule_id', 'atom_index', 'x', 'y', 'z']], left_on = ['molecule_id', 'atom_index_1'], right_on = ['molecule_id', 'atom_index'], how = 'left')[['x', 'y', 'z']]

distance = (bond_xyz_0 - bond_xyz_1).values

In [ ]:
distance = np.sqrt(np.square(distance).sum(axis = 1))

In [ ]:
distance.shape

In [ ]:
all_bond_features['distance'] = distance

In [ ]:
BOND_FEATURES_COLUMNS += ['distance']

## Add bond coordinates

In [ ]:
coordinates = (bond_xyz_0 + bond_xyz_1) / 2
all_bond_features['x_c'] = coordinates['x']
all_bond_features['y_c'] = coordinates['y']
all_bond_features['z_c'] = coordinates['z']

In [ ]:
all_bond_features['x_0'] = bond_xyz_0['x']
all_bond_features['y_0'] = bond_xyz_0['y']
all_bond_features['z_0'] = bond_xyz_0['z']
all_bond_features['x_1'] = bond_xyz_1['x']
all_bond_features['y_1'] = bond_xyz_1['y']
all_bond_features['z_1'] = bond_xyz_1['z']

## Add edge_index

In [ ]:
all_bond_features['edge_index'] = all_bond_features[['molecule_id']].groupby('molecule_id')['molecule_id'].transform(lambda x : np.arange(x.shape[0], dtype = int))

## Add descriptors

In [ ]:
atom_descriptors = all_atom_features[['molecule_id', 'atom', 'x', 'y', 'z']].set_index('molecule_id')
atom_descriptors['index'] = np.arange(atom_descriptors.shape[0], dtype = int)

bond_descriptors = all_bond_features[['molecule_id', 'atom_index_0', 'atom_index_1', 'type', 'scalar_coupling_constant', 'edge_index', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1']].set_index('molecule_id')
bond_descriptors['index'] = np.arange(bond_descriptors.shape[0], dtype = int)

## Use transformers

In [ ]:
print("global_transforms = [")
for c in all_global_features.columns:
    unique_count = all_global_features[c].nunique()
    if c in ['molecule_id', 'atom_index_0', 'atom_index_1', 'x', 'y', 'z', 'dataset', 'scalar_coupling_constant', 'atom_index']:
        print(f"    (\"{c}\", \"null\", {unique_count}),")
    else:
        if unique_count == 1:
            print(f"    (\"{c}\", \"null\", {unique_count}),")
        elif unique_count < 50:
            print(f"    (\"{c}\", \"one_hot\", {unique_count}),")
        else:
            print(f"    (\"{c}\", \"bins&value\", {unique_count}),")
print("]")
print()

print("bond_transforms = [")
for c in all_bond_features.columns:
    unique_count = all_bond_features[c].nunique()
    if c in ['molecule_id', 'atom_index_0', 'atom_index_1', 'x', 'y', 'z', 'dataset', 'scalar_coupling_constant', 'atom_index', 'edge_index']:
        print(f"    (\"{c}\", \"null\", {unique_count}),")
    else:
        if unique_count == 1:
            print(f"    (\"{c}\", \"null\", {unique_count}),")
        elif unique_count < 50:
            print(f"    (\"{c}\", \"one_hot\", {unique_count}),")
        else:
            print(f"    (\"{c}\", \"bins&value\", {unique_count}),")
print("]")
print()
print("atom_transforms = [")
for c in all_atom_features.columns:
    unique_count = all_atom_features[c].nunique()
    if c in ['molecule_id', 'atom_index_0', 'atom_index_1', 'x', 'y', 'z', 'dataset', 'scalar_coupling_constant', 'atom_index']:
        print(f"    (\"{c}\", \"null\", {unique_count}),")
    else:
        if unique_count == 1:
            print(f"    (\"{c}\", \"null\", {unique_count}),")
        elif unique_count < 50:
            print(f"    (\"{c}\", \"one_hot\", {unique_count}),")
        else:
            print(f"    (\"{c}\", \"bins&value\", {unique_count}),")
print("]")
print()

In [ ]:
global_transforms = [
    ("molecule_id", "null", 130775),
    ("NumAtoms", "one_hot", 26),
    ("NumBonds", "one_hot", 27),
    ("NumHvyAtoms", "one_hot", 9),
    ("NumResidues", "null", 1),
    ("NumRotors", "one_hot", 7),
    ("GetMolWt", "bins&value", 977),
    ("GetEnergy", "null", 1),
    ("GetExactMass", "bins&value", 698),
    ("GetTotalCharge", "null", 1),
    ("GetTotalSpinMultiplicity", "one_hot", 7),
    ("IsChiral", "one_hot", 2),
    ("NumConformers", "null", 1),
]

bond_transforms = [
    ("molecule_id", "null", 130775),
    ("IsBond", "one_hot", 2),
    ("GetBondOrder", "one_hot", 4),
    ("GetEquibLength", "one_hot", 44),
    ("GetLength", "bins&value", 2439794),
    ("IsAromatic", "one_hot", 2),
    ("IsInRing", "one_hot", 2),
    ("IsRotor", "one_hot", 2),
    ("IsAmide", "one_hot", 2),
    ("IsPrimaryAmide", "one_hot", 2),
    ("IsSecondaryAmide", "one_hot", 2),
    ("IsTertiaryAmide", "one_hot", 2),
    ("IsEster", "one_hot", 2),
    ("IsCarbonyl", "one_hot", 2),
    ("IsSingle", "one_hot", 2),
    ("IsDouble", "one_hot", 2),
    ("IsTriple", "one_hot", 2),
    ("IsClosure", "one_hot", 2),
    ("IsUp", "null", 1),
    ("IsDown", "null", 1),
    ("IsCisOrTrans", "null", 1),
    ("IsDoubleBondGeometry", "one_hot", 2),
    ("atom_index_0", "null", 28),
    ("atom_index_1", "null", 28),
    ("type", "one_hot", 9),
    ("scalar_coupling_constant", "null", 2182935),
    ("dataset", "null", 3),
    ("dist_inv2", "bins&value", 20655504),
    ("distance_closest_0", "bins&value", 1502312),
    ("distance_closest_1", "bins&value", 1519517),
    ("distance_farthest_0", "bins&value", 2084472),
    ("distance_farthest_1", "bins&value", 2050387),
    ("cos_c0_c1", "bins&value", 15524389),
    ("cos_f0_f1", "bins&value", 20433739),
    ("cos_c0_f0", "bins&value", 2227874),
    ("cos_c1_f1", "bins&value", 2227874),
    ("cos_center0_center1", "bins&value", 20655504),
    ("cos_c0", "bins&value", 19863686),
    ("cos_c1", "bins&value", 19413981),
    ("cos_f0", "bins&value", 19214284),
    ("cos_f1", "bins&value", 20050530),
    ("cos_center0", "bins&value", 20655496),
    ("cos_center1", "bins&value", 20655490),
    ("molecule_atom_index_0_dist_mean", "bins&value", 2227882),
    ("molecule_atom_index_0_dist_mean_diff", "bins&value", 20524789),
    ("molecule_atom_index_0_dist_min", "bins&value", 2227875),
    ("molecule_atom_index_0_dist_min_diff", "bins&value", 18427633),
    ("molecule_atom_index_0_dist_std", "bins&value", 2097108),
    ("molecule_atom_index_1_dist_mean", "bins&value", 2227882),
    ("molecule_atom_index_1_dist_mean_diff", "bins&value", 20524777),
    ("molecule_atom_index_1_dist_min", "bins&value", 2227874),
    ("molecule_atom_index_1_dist_min_diff", "bins&value", 18427629),
    ("molecule_atom_index_1_dist_std", "bins&value", 2097108),
    ("molecule_type_dist_mean", "bins&value", 942061),
    ("molecule_type_dist_mean_diff", "bins&value", 20581261),
    ("distance", "bins&value", 20655504),
    ("edge_index", "null", 406),
]

atom_transforms = [
    ("molecule_id", "null", 130775),
    ("GetFormalCharge", "null", 1),
    ("GetSpinMultiplicity", "one_hot", 3),
    ("GetAtomicMass", "one_hot", 5),
    ("GetExactMass", "one_hot", 5),
    ("GetAtomicNum", "one_hot", 5),
    ("GetValence", "one_hot", 4),
    ("GetHyb", "one_hot", 4),
    ("GetImplicitValence", "one_hot", 4),
    ("GetHvyValence", "one_hot", 5),
    ("GetHeteroValence", "one_hot", 4),
    ("GetPartialCharge", "bins&value", 1754625),
    ("CountFreeOxygens", "one_hot", 3),
    ("ImplicitHydrogenCount", "one_hot", 2),
    ("ExplicitHydrogenCount", "one_hot", 5),
    ("MemberOfRingCount", "one_hot", 5),
    ("MemberOfRingSize", "one_hot", 8),
    ("CountRingBonds", "one_hot", 4),
    ("SmallestBondAngle", "bins&value", 1073315),
    ("AverageBondAngle", "bins&value", 1070999),
    ("BOSum", "one_hot", 5),
    ("HasResidue", "null", 1),
    ("IsAromatic", "one_hot", 2),
    ("IsInRing", "one_hot", 2),
    ("IsHeteroatom", "one_hot", 2),
    ("IsNotCorH", "one_hot", 2),
    ("IsCarboxylOxygen", "one_hot", 2),
    ("IsPhosphateOxygen", "null", 1),
    ("IsSulfateOxygen", "null", 1),
    ("IsNitroOxygen", "one_hot", 2),
    ("IsAmideNitrogen", "one_hot", 2),
    ("IsPolarHydrogen", "one_hot", 2),
    ("IsNonPolarHydrogen", "one_hot", 2),
    ("IsAromaticNOxide", "null", 1),
    ("IsChiral", "one_hot", 2),
    ("IsAxial", "one_hot", 2),
    ("IsHbondAcceptor", "one_hot", 2),
    ("IsHbondDonor", "one_hot", 2),
    ("IsHbondDonorH", "one_hot", 2),
    ("HasAlphaBetaUnsat", "one_hot", 2),
    ("HasNonSingleBond", "one_hot", 2),
    ("HasSingleBond", "one_hot", 2),
    ("HasDoubleBond", "one_hot", 2),
    ("HasAromaticBond", "null", 1),
    ("x", "null", 2358441),
    ("y", "null", 2358364),
    ("z", "null", 2358421),
    ("atom", "one_hot", 5),
    ("atom_index", "null", 29)
]

### compute feature names

In [ ]:
names = [[] for _ in range(6)]

In [ ]:
for group_i, feature_group in enumerate([global_transforms, atom_transforms, bond_transforms]):
    for transform in feature_group:
        transform_type = transform[1]
        transform_name = transform[0]
        if transform_type == 'bins&value':
            names[group_i].append(f'{transform_name}_embedding')
            names[group_i + 3].append(f'{transform_name}_numeric')
        elif transform_type == 'one_hot':
            names[group_i].append(f'{transform_name}_embedding')

In [ ]:
# Add atom xyz
names[4] += ['x_numeric', 'y_numeric', 'z_numeric']
# Add acsf
names[4] += [f'acsf_{i}' for i in range(80)]
# Add vector xyz
names[5] += ['x_numeric', 'y_numeric', 'z_numeric']

In [ ]:
[len(e) for e in names]

In [ ]:
import pickle
with open('processed_data/names_117.pkl', 'wb') as f:
    pickle.dump(names, f)

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder, StandardScaler
from scipy.sparse import hstack, csr_matrix

## Separate train and test

In [ ]:
dataset = partial_edges[['molecule_name', 'molecule_id','dataset']].drop_duplicates('molecule_name')

In [ ]:
dataset['molecule_id'].max()

In [ ]:
dataset['molecule_id'].value_counts().value_counts()

In [ ]:
dataset.shape

## Set types ids

In [ ]:
encoder = OrdinalEncoder()
bond_descriptors['type_id'] = 0
bond_descriptors.loc[bond_descriptors['type'] != 'VOID', 'type_id'] = encoder.fit_transform(bond_descriptors.loc[bond_descriptors['type'] != 'VOID', ['type']])[:, 0]

# Add bond distance

In [ ]:
all_bond_features.head()

In [ ]:
import networkx as nx

In [ ]:
all_bond_features.shape

In [ ]:
graph_atoms = all_atom_features[['molecule_id', 'atom_index']].groupby("molecule_id")
graph_bonds_directed = all_bond_features.loc[(all_bond_features['IsBond'] == 1), ['molecule_id', 'atom_index_0', 'atom_index_1', 'edge_index']].groupby('molecule_id')
graph_edges_directed = all_bond_features[['molecule_id', 'atom_index_0', 'atom_index_1', 'edge_index']].groupby('molecule_id')

In [ ]:
bond_distance = []

for molecule_id in tqdm.tqdm_notebook(graph_atoms.groups):
    nodes = graph_atoms.get_group(molecule_id)['atom_index'].values
    bonds = graph_bonds_directed.get_group(molecule_id)[['atom_index_0', 'atom_index_1']].values
    edges = graph_edges_directed.get_group(molecule_id)[['atom_index_0', 'atom_index_1']].values
    
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(bonds)
    
    for start, end in edges:
        d = nx.dijkstra_path_length(G, start, end)
        bond_distance.append(d)
    

In [ ]:
len(bond_distance)

In [ ]:
bond_descriptors['bond_distance'] = bond_distance

In [ ]:
nx.draw_networkx(G, figsize = (10, 5))

# Add edge connectivity

In [ ]:
#@numba.njit
def compute_edge_connectivity(edges_atom_index, edges_indexes, vector_edges, bonds_atom_index, bonds_indexes):
    edges_connectivity = []
    edges_link_vectors = []
    
    where_are_the_edges_connected = {}
    which_is_the_first_atom = {}
    
    for bond_i in range(bonds_atom_index.shape[0]):
        bond_index = bonds_indexes[bond_i]
        
        atom_index_0 = bonds_atom_index[bond_i, 0]
        
        if atom_index_0 in where_are_the_edges_connected:
            where_are_the_edges_connected[atom_index_0] = np.concatenate((where_are_the_edges_connected[atom_index_0], np.array([bond_index])))
        else:
            where_are_the_edges_connected[atom_index_0] = np.array([bond_index])
            
        which_is_the_first_atom[bond_index] = atom_index_0
        
        atom_index_1 = bonds_atom_index[bond_i, 1]
        
        if atom_index_1 in where_are_the_edges_connected:
            where_are_the_edges_connected[atom_index_1] = np.concatenate((where_are_the_edges_connected[atom_index_1], np.array([bond_index])))
        else:
            where_are_the_edges_connected[atom_index_1] = np.array([bond_index])
    
    for edge_i in range(edges_atom_index.shape[0]):
        atom_index_0 = edges_atom_index[edge_i, 0]
        atom_index_1 = edges_atom_index[edge_i, 1]
        edge_index = edges_indexes[edge_i]
        
        edges_connected_to_atom_0 = where_are_the_edges_connected[atom_index_0]
        edges_connected_to_atom_1 = where_are_the_edges_connected[atom_index_1]
        
        for edge in edges_connected_to_atom_0:
            if edge != edge_i:
                vector_0 = - vector_edges[edge_i]
                
                bond_atom_index_0 = which_is_the_first_atom[edge]
                if bond_atom_index_0 == atom_index_0:
                    vector_1 = vector_edges[edge]
                else:
                    vector_1 = - vector_edges[edge]
                
                edges_connectivity.append((edge_i, edge))
                edges_link_vectors.append(np.concatenate((vector_0, vector_1)).reshape(1, -1))
                
        for edge in edges_connected_to_atom_1:
            if edge != edge_i:
                vector_0 = vector_edges[edge_i]
            
                bond_atom_index_0 = which_is_the_first_atom[edge]
                if bond_atom_index_0 == atom_index_1:
                    vector_1 = vector_edges[edge]
                else:
                    vector_1 = - vector_edges[edge]
            
                edges_connectivity.append((edge_i, edge))
                edges_link_vectors.append(np.concatenate((vector_0, vector_1)).reshape(1, -1))
                
    return np.array(edges_connectivity), edges_link_vectors

In [ ]:
graph_bonds_directed = all_bond_features.loc[(all_bond_features['IsBond'] == 1), ['molecule_id', 'atom_index_0', 'atom_index_1', 'edge_index']].groupby('molecule_id')
graph_edges_directed = all_bond_features[['molecule_id', 'atom_index_0', 'atom_index_1', 'edge_index', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1']].groupby('molecule_id')

In [ ]:
edges_connectivity = []
all_vectors = []
molecules_ids = []
for molecule_id in tqdm.tqdm_notebook(graph_atoms.groups):
    bonds = graph_bonds_directed.get_group(molecule_id)
    edges = graph_edges_directed.get_group(molecule_id)
    
    vectors_edges = edges[['x_1', 'y_1', 'z_1']].values - edges[['x_0', 'y_0', 'z_0']].values
    edge_connectivity, vectors = compute_edge_connectivity(edges[['atom_index_0', 'atom_index_1']].values, edges['edge_index'].values, vectors_edges, bonds[['atom_index_0', 'atom_index_1']].values, bonds['edge_index'].values)
    vectors = np.concatenate(vectors, axis = 0)
    
    all_vectors.append(vectors)
    edges_connectivity.append(edge_connectivity)
    molecules_ids.append(np.full((edge_connectivity.shape[0], 1), molecule_id))

In [ ]:
edges_connectivity = np.concatenate(edges_connectivity, axis = 0)
molecules_ids = np.concatenate(molecules_ids, axis = 0)
all_vectors = np.concatenate(all_vectors, axis = 0)
edges_connectivity = np.concatenate([molecules_ids, edges_connectivity, all_vectors], axis = 1)
edges_connectivity = pd.DataFrame(edges_connectivity, columns = ['molecule_id', 'edge_index_0', 'edge_index_1', 'vx_0', 'vy_0', 'vz_0', 'vx_1', 'vy_1', 'vz_1'])

In [ ]:
edges_connectivity.shape

In [ ]:
edges_connectivity.head()

# Add rings

In [ ]:
@numba.njit
def compute_edge_cycle_connectivity(bonds_atom_index, bonds_indexes, cycles):
    edge_map = {}
    
    for bond_i in range(bonds_atom_index.shape[0]):
        edge_map[(bonds_atom_index[bond_i, 0], bonds_atom_index[bond_i, 1])] = bonds_indexes[bond_i]
    
    edges_connectivity = []
    cycle_id = []
    
    for cycle_i in range(len(cycles) - 1):
        cycle = cycles[cycle_i]
        
        for atom_i in range(cycle.shape[0]):
            if atom_i == cycle.shape[0] - 1:
                atom_index_0 = cycle[0]
                atom_index_1 = cycle[atom_i]
            else:
                atom_index_0 = cycle[atom_i]
                atom_index_1 = cycle[atom_i + 1]
                
            atom_index_0, atom_index_1 = min(atom_index_0, atom_index_1), max(atom_index_0, atom_index_1)
            
            edge_index = edge_map[(atom_index_0, atom_index_1)]
            
            edges_connectivity.append(edge_index)
            cycle_id.append(cycle_i)
        
                
    return np.array(edges_connectivity), np.array(cycle_id)

In [ ]:
import networkx as nx

In [ ]:
graph_atoms = all_atom_features[['molecule_id', 'atom_index']].groupby("molecule_id")
graph_bonds_directed = all_bond_features.loc[(all_bond_features['IsBond'] == 1), ['molecule_id', 'atom_index_0', 'atom_index_1', 'edge_index']].groupby('molecule_id')
graph_edges_directed = all_bond_features[['molecule_id', 'atom_index_0', 'atom_index_1', 'edge_index']].groupby('molecule_id')

In [ ]:
cycle_edges_connectivity = []
cycle_ids = []
molecule_ids = []
for molecule_id in tqdm.tqdm_notebook(graph_atoms.groups):
    nodes = graph_atoms.get_group(molecule_id)['atom_index'].values
    graph_bonds = graph_bonds_directed.get_group(molecule_id)[['atom_index_0', 'atom_index_1']].values
    bonds = graph_bonds_directed.get_group(molecule_id)
    
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(graph_bonds)
    
    cycles = nx.cycle_basis(G)
    cycles = [np.array(cycle) for cycle in cycles] + [np.array([0])]
    
    #if len(cycles) == 4:
    #    break
    
    edge_connectivity, cycle_id = compute_edge_cycle_connectivity(bonds[['atom_index_0', 'atom_index_1']].values, bonds['edge_index'].values, cycles)
    cycle_edges_connectivity.append(edge_connectivity)
    cycle_ids.append(cycle_id)
    molecule_ids.append(np.full_like(cycle_id, molecule_id))

In [ ]:
cycle_edges_connectivity = np.concatenate(cycle_edges_connectivity).reshape(-1, 1)
cycle_ids = np.concatenate(cycle_ids).reshape(-1, 1)
molecule_ids = np.concatenate(molecule_ids).reshape(-1, 1)
cycles = np.concatenate([molecule_ids, cycle_edges_connectivity, cycle_ids], axis = 1)

cycles = pd.DataFrame(cycles, columns = ['molecule_id', 'edge_index', 'cycle_id'])

In [ ]:
cycles.head(10)

In [ ]:
cycles.shape

In [ ]:
nx.draw_networkx(G, figsize = (10, 5))

## Save

In [ ]:
atom_descriptors.to_pickle(script_dir + '/processed_data/atom_descriptors_117.pkl')
bond_descriptors.to_pickle(script_dir + '/processed_data/bond_descriptors_117.pkl')

In [ ]:
dataset.to_pickle(script_dir + '/processed_data/dataset_descriptor_117.pkl')

In [ ]:
cycles.to_pickle(script_dir + '/processed_data/cycles_117.pkl')
edges_connectivity.to_hdf(script_dir + '/processed_data/edges_connectivity_117.pkl', 'data')

In [ ]:
e = pd.read_hdf(script_dir + '/processed_data/edges_connectivity_117.pkl', 'data')

In [ ]:
e.head()

### Empty memory

In [ ]:
Out = {}

In [ ]:
print([e for e in list(locals().keys()) if e[0] != '_'])

In [ ]:
del nodes
del train
del test
del partial_edges
del atom_count
del all_edges
del global_features
del atom_features
del bond_features
del indexed_nodes
del positions
del molecules_index
del merge_1
del merge_2
del c
del bond_xyz_0
del bond_xyz_1
del distance
del unique_count
del atom_descriptors
del bond_descriptors
del dataset

In [ ]:
import gc
gc.collect()

In [ ]:
from numpy import savez_compressed, load

In [ ]:
from sklearn.preprocessing import QuantileTransformer

In [ ]:
def transform(to_transform, transform_infos, filename):
    output_transform_numeric = []
    output_transform_embeddings = []

    for transform_info in transform_infos:
        column = transform_info[0]
        if transform_info[1] == 'one_hot':
            encoder = OrdinalEncoder()
            transform = encoder.fit_transform(to_transform[[column]].astype(str))
            output_transform_embeddings.append(transform)
            print(transform_info, transform.shape[1])

        if transform_info[1] == 'bins&value':
            encoder = KBinsDiscretizer(n_bins = 25, encode = 'ordinal')
            transform = encoder.fit_transform(to_transform[[column]])
            output_transform_embeddings.append(transform)
            print(transform_info, transform.shape[1])
            
            encoder = QuantileTransformer()
            transform = encoder.fit_transform(to_transform[[column]])
            output_transform_numeric.append(transform)
            print(transform_info, transform.shape[1])

        if transform_info[1] == 'identity':
            encoder = QuantileTransformer()
            transform = encoder.fit_transform(to_transform[[column]])
            output_transform_numeric.append(transform)
            print(transform_info, transform.shape[1])

    base_index = 0
    for column in output_transform_embeddings:
        max_index = column.max() + 1
        column += base_index
        base_index += max_index
            
    output_transform_numeric = np.concatenate(output_transform_numeric, axis = 1)
    output_transform_embeddings = np.concatenate(output_transform_embeddings, axis = 1).astype(np.int32)

    savez_compressed(filename, embeddings = output_transform_embeddings, numeric = output_transform_numeric)

In [ ]:
transform(all_global_features, global_transforms, script_dir + '/processed_data/global_116.npz')
del all_global_features, global_transforms
gc.collect()

transform(all_atom_features, atom_transforms, script_dir + '/processed_data/atom_116.npz')
del all_atom_features, atom_transforms
gc.collect()

transform(all_bond_features, bond_transforms, script_dir + '/processed_data/bond_116.npz')
del all_bond_features, bond_transforms
gc.collect()

In [ ]:
import h5py

import numpy as np

for type_file in ['atom', 'global', 'bond']:
    array_group = load(f'{script_dir}/processed_data/{type_file}_116.npz')
    h5_file = h5py.File(f'{script_dir}/processed_data/{type_file}_116.h5')
    
    h5_file.create_dataset('numeric', data = array_group.get('numeric').astype(np.float32))
    h5_file.create_dataset('embeddings', data = array_group.get('embeddings').astype(np.int32))
    
    h5_file.close()